In [30]:
import time
from zmqRemoteApi import RemoteAPIClient
import numpy as np 


class parabolicBlends():
    def __init__(self):
        pass 
    def generateCurve(self,x,y,a):
        assert len(x)==len(y)
        # print(x)
        self.n=len(x)
        self.x=x 
        self.y=y
        self.abs_a=np.abs(a)
        # number of via points 
        self.k=[0]
        for i in range(self.n-1):
            self.k+=[(self.y[i+1]-self.y[i])/(self.x[i+1]-self.x[i])]
        self.k+=[0]
        self.a=[]
        self.b=[]
        self.c=[]
        self.l=[]
        self.r=[]
        self.intervals=[]
        for i in range(self.n):
            l,r,a,b,c=parabolicBlends.getParabolic([self.x[i],self.y[i],self.k[i],self.k[i+1]],self.abs_a)
            self.a+=[a]
            self.b+=[b]
            self.c+=[c]
            self.l+=[l]
            self.r+=[r]
            self.intervals+=[l,r]
        self.intervals=np.array(self.intervals)
        # print('x',self.x)
        # print('y',self.y)
        # print('k',self.k)
        # print('intervals',self.intervals)
    def getValue(self,x):
        i=np.searchsorted(self.intervals,x)
        if i==0 :return self.y[0]
        # then the x was in the i-1~i+1
        if i%2==0:
            return self.y[int(i/2-1)]+(x-self.x[int(i/2-1)])*self.k[int(i/2)]
        if i%2==1:
            return self.a[int(i/2)]*x**2+self.b[int(i/2)]*x+self.c[int(i/2)]
    def calculate(self,x):
        res=[]
        for i in range(len(x)):
            res+=[self.getValue(x[i])]
        return np.array(res)
    @staticmethod
    def getParabolic(para,a=30):
        assert len(para)==4
        # y=ax2+bx+c 
        # with the tangent 
        # y=k1(x-x0)+y0
        # y=k2(x-x0)+y0
        x0,y0,k1,k2=para 
        if (k1>k2):
            a=-a
        x1=(k1-k2)/4/a+x0 
        x2=(k2-k1)/4/a+x0
        b=(k1+k2)/2-2*a*x0 
        c=y0+a*x0**2-(k1+k2)/2*x0+((k1-k2)**2/16/a)
        # print((x1,x2,a,b,c))
        return (x1,x2,a,b,c)
def LinearParabolicBlends(theta,timePoints,a=400,itersPerSec=50):
    CureveGenerator=parabolicBlends()
    # time[0]=0 time[-1]=2
    theta=np.array(theta).T
    # theta timepoints are 6*n n equals to the interval points 
    timePoints=np.array(timePoints)
    # if timePoints.shape[0]<6:
    #     timePoints=timePoints[0].repeat(6, axis=0)
    # print(timePoints)
    # time_interval=np.array(time_interval)
    totalIters=int(itersPerSec*(timePoints[-1]-timePoints[0]))
    dt=1/itersPerSec
    x=np.linspace(timePoints[0],timePoints[-1],totalIters)
    t=0
    q=[]
    for i in range(len(theta)):
        # print((timePoints,theta[i],a))
        CureveGenerator.generateCurve(timePoints,theta[i],a)
        q+=[CureveGenerator.calculate(x)]
    # self.control(q,totalIters,dt)
    return q
class TianHe():
    def __init__(self) -> None:
        self.dt=0.02 
        self.client=RemoteAPIClient()
        self.client.setStepping(True)
        self.sim = self.client.getObject('sim')

        self.joint_nums=7
        print ('Connected to ZMQ API server')
        # 初始化机械臂关节
        self.retValue = []
        self.jointHandle = []
        # self.Foot = self.sim.getObject("/Foot1")
        self.foot=1
        self.root=self.sim.getObject("/Foot1")
        while(self.sim.getObjectParent(self.root)!=-1):
            self.root=self.sim.getObjectParent(self.root)
        self.objectTree=[self.root]
        while(self.objectTree[-1]!=-1):
            self.objectTree.append(self.sim.getObjectChild(self.objectTree[-1],0))
        self.objectTree.pop()
        print(self.objectTree)
        for i in range(7):
            jointName = "/Joint" + str(i+1)
            joint = self.sim.getObject( jointName)
            # print(joint)
            # self.retValue.append(ret)
            self.jointHandle.append(joint)
            self.shape= self.sim.getObject('/Shape')
        self.generateCurve=LinearParabolicBlends
        self.sim.startSimulation()
        self.setJoint([0,0,0,0,0,0,0])
        
        print(self.jointHandle)
        pass
    def setJoint(self,q):
        for ith in range(self.joint_nums):
            self.sim.setJointPosition(self.jointHandle[ith],q[ith])
        pass 
    def getJoint(self):
        q=[]
        for ith in range(self.joint_nums):
            q.append(self.sim.getJointPosition(self.jointHandle[ith]))
        return np.array(q)
        pass 
    def stop(self):
        self.sim.stopSimulation()

    def changeFoot(self):
        self.sim.setObjectParent(self.objectTree[-1],-1,True)
        for i in range(len(self.objectTree)-1,0,-1):
            self.sim.setObjectParent(self.objectTree[i-1],self.objectTree[i],True)
            # print(self.objectTree[i-1],self.objectTree[i])
        self.objectTree=self.objectTree[::-1]
        self.jointHandle=self.jointHandle[::-1]
        if(self.foot==1):
            self.foot==2
        else:
            self.foot==1
        pass 
    def control(self,q=[[0,0,0,0,0,0,0]],t=[0]):
        qs=self.generateCurve(q,t)
        qs=np.array(qs).T
        print(len(qs))
        for i in qs:
            self.setJoint(i)
            time.sleep(self.dt)
        pass
    def pace(self,d,r1,r2,theta)->list:
        # return 7 angle 
        # inverse
        pass 


Robot=TianHe()
L=3

# q1=0
d=np.sqrt(L**2+4*L+8)
# d=2+L*np.cos(q1)
q4=np.arcsin(d/10.4)
q1=np.arctan2(1,5.2*np.sin(q4/2)-1)

q1=q1/np.pi*180+5
q4=q4/np.pi*180
print(d,q1,q4)
a1,a2,a3,a4=q1+90,90,q4+90,-180+q4*2
q1=np.array([ 
    [0,0,0,0,0,0,0],
    [a1,a2,a3,a4,-a3,-a2,-a1],
])
q1=q1/180*np.pi
t=[0,2]
Robot.control(q1,t)
Robot.changeFoot()
a1=a1-180
q2=np.array([ 
    Robot.getJoint()*(-180)/np.pi,
    Robot.getJoint()*(-180)/np.pi+np.array([0,0,0,-5,-10,0,0]),
    Robot.getJoint()*(-180)/np.pi+np.array([-180,0,0,-5,-10,0,0]),
    Robot.getJoint()*(-180)/np.pi+np.array([-180,0,0,0,0,0,0]),
    # [a1,a2,a3,a4,-a3,-a2,-a1][::-1],
])
t=[0,1,2,3]
q2=q2/180*np.pi+2*np.array(Robot.getJoint())
Robot.control(q2,t)

# time.sleep(5)
Robot.changeFoot()
joint_ori2=Robot.getJoint()
# q3=np.array([ 
#     # Robot.getJoint()*(-180)/np.pi,
#     [0,0,0,0,0,0,0],
#     # [60,30,10,-110,-45,-100,90],
#     joint_ori2*(-180)/np.pi+np.array([60,30,12,-110,-45,-100,-28][::-1]),
# ])
# t=[0,1]
# q3=q3/180*np.pi+joint_ori2
# Robot.control(q3,t)
# time.sleep(5)



# q3=np.array([ 
#     # Robot.getJoint()*(-180)/np.pi,
#     [0,0,0,0,0,0,0]+joint_ori2*(-180)/np.pi,
#     # [60,30,10,-110,-45,-100,90],
#     # joint_ori2*(-180)/np.pi+np.array([60,30,12,-110,-45,-100,360-28][::-1]),
#     # np.array([60,30,12,-110,-45,-100,-28][::-1]),
#     # [-20,-36,0,-20,0,0,0]+joint_ori2*(-180)/np.pi,
#     [0,0,0,-20,10,0,0]+joint_ori2*(-180)/np.pi,
#     [0,0,0,-20,-160,160,0]+joint_ori2*(-180)/np.pi,
#     [-60,-40,-5,10,-160,160,0]+joint_ori2*(-180)/np.pi,
#     # np.array([60,30,12,110,-45,-100,-28-90][::-1]),
# ])
# t=[0,0.5,1,2]
# q3=q3/180*np.pi
# Robot.control(q3,t)
# time.sleep(5)
# Robot.stop()

q3=np.array([ 
    # Robot.getJoint()*(-180)/np.pi,
    [0,0,0,0,0,0,0]+joint_ori2*(180)/np.pi,
    [0,0,0,20,-10,0,0]+joint_ori2*(180)/np.pi,
    [0,0,0,20,160,-160,0]+joint_ori2*(180)/np.pi,
    [60.542,40.878,5.210,-12.674,160.057,-160.012,-45]+joint_ori2*(180)/np.pi,
])
t=[0,0.5,1,2]
q3=q3/180*np.pi
Robot.control(q3,t)
time.sleep(5)
Robot.stop()

Connected to ZMQ API server
[15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[16, 18, 20, 22, 24, 26, 28]
5.385164807134504 73.31074509903694 31.185083756689227
100
150
100


In [2]:
# from scipy import interpolate
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker


In [5]:
def draw(q):
    
    # plt.figure(figsize=(15,10))
    y=q.copy()
    t=[0,(len(y[0])-1)*0.02]
    n=len(y[0])
    x=np.linspace(0,t[-1],n)
    plt.figure(figsize=(14,7))
    plt.subplot(3,1,1)
    plt.plot(x,y[0],label=r'$joint_1$')#,linestyle=':')
    plt.plot(x,y[1],label=r'$joint_2$')#,linestyle=':')
    plt.plot(x,y[2],label=r'$joint_3$')#,linestyle=':')
    plt.plot(x,y[3],label=r'$joint_4$')#,linestyle=':')
    plt.plot(x,y[4],label=r'$joint_5$')#,linestyle=':')
    plt.plot(x,y[5],label=r'$joint_6$')#,linestyle=':')
    plt.plot(x,y[6],label=r'$joint_7$')#,linestyle=':')
    # plt.plot(x,y[5],label=r'$joint_6$')#,linestyle=':')
    plt.xlabel('t/s',fontsize=14)
    plt.ylabel(r'$\theta/^{\circ}$',fontsize=14)
    plt.grid(linestyle='-.',which='both')
    plt.legend(loc=2,ncol=2)


    plt.subplot(3,1,2)
    for i in range(7):
        y[i]=(y[i][1:]-y[i][:-1])/0.02/2.5
    plt.plot(x[1:],y[0],label=r'$joint_1$')#,linestyle=':')
    plt.plot(x[1:],y[1],label=r'$joint_2$')#,linestyle=':')
    plt.plot(x[1:],y[2],label=r'$joint_3$')#,linestyle=':')
    plt.plot(x[1:],y[3],label=r'$joint_4$')#,linestyle=':')
    plt.plot(x[1:],y[4],label=r'$joint_5$')#,linestyle=':')
    plt.plot(x[1:],y[5],label=r'$joint_6$')#,linestyle=':')
    plt.plot(x[1:],y[6],label=r'$joint_7$')#,linestyle=':')
    # plt.plot(x[1:],y[5],label=r'$joint_6$')#,linestyle=':')
    plt.xlabel('t/s',fontsize=14)
    plt.ylabel(r'$\alpha/( ^{\circ} /s)$',fontsize=14)
    plt.grid(linestyle='-.',which='both')
    plt.legend(loc=2,ncol=2)

    plt.subplot(3,1,3)
    for i in range(7):
        y[i]=(y[i][1:]-y[i][:-1])/0.02/2
    plt.plot(x[2:],y[0],label=r'$joint_1$')#,linestyle=':')
    plt.plot(x[2:],y[1],label=r'$joint_2$')#,linestyle=':')
    plt.plot(x[2:],y[2],label=r'$joint_3$')#,linestyle=':')
    plt.plot(x[2:],y[3],label=r'$joint_4$')#,linestyle=':')
    plt.plot(x[2:],y[4],label=r'$joint_5$')#,linestyle=':')
    plt.plot(x[2:],y[5],label=r'$joint_6$')#,linestyle=':')
    plt.plot(x[2:],y[6],label=r'$joint_7$')#,linestyle=':')
    # plt.plot(x[2:],y[5],label=r'$joint_6$')#,linestyle=':')
    plt.xlabel('t/s',fontsize=14)
    plt.ylabel(r'$\beta/( ^{\circ} /s^2)$',fontsize=14)
    plt.grid(linestyle='-.',which='both')
    plt.legend(loc=2,ncol=2)


    plt.show()

In [28]:
Robot.stop()

In [16]:

Robot=TianHe()
L=3

# q1=0
d=np.sqrt(L**2+4*L+8)
# d=2+L*np.cos(q1)
q4=np.arcsin(d/10.4)
q1=np.arctan2(1,5.2*np.sin(q4/2)-1)

q1=q1/np.pi*180+5
q4=q4/np.pi*180
print(d,q1,q4)
a1,a2,a3,a4=q1+90,90,q4+90,-180+q4*2
q1=np.array([ 
    [0,0,0,0,0,0,0],
    [a1,a2,a3,a4,-a3,-a2,-a1],
])
q1=q1/180*np.pi
t=[0,2]
Robot.control(q1,t)
# qs=Robot.generateCurve(q1,t)
# qs=np.array(qs).T
# draw(qs)


Connected to ZMQ API server
[15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]
[16, 18, 20, 22, 24, 26, 28]
5.385164807134504 73.31074509903694 31.185083756689227
100


In [17]:

Robot.changeFoot()
a1=a1-180
q2=np.array([ 
    Robot.getJoint()*(-180)/np.pi,
    Robot.getJoint()*(-180)/np.pi+np.array([0,0,0,-5,-10,0,0]),
    Robot.getJoint()*(-180)/np.pi+np.array([-180,0,0,-5,-10,0,0]),
    Robot.getJoint()*(-180)/np.pi+np.array([-180,0,0,0,0,0,0]),
    # [a1,a2,a3,a4,-a3,-a2,-a1][::-1],
])
t=[0,1,2,3]
q2=q2/180*np.pi+2*np.array(Robot.getJoint())
Robot.control(q2,t)
qs=Robot.generateCurve(q2,t)
# qs=np.array(qs).T
# time.sleep(5)
# draw(qs)
Robot.changeFoot()
joint_ori2=Robot.getJoint()

150


In [27]:

q3=np.array([ 
    # Robot.getJoint()*(-180)/np.pi,
    [0,0,0,0,0,0,0]+joint_ori2*(180)/np.pi,
    [0,0,0,20,-10,0,0]+joint_ori2*(180)/np.pi,
    [0,0,0,20,160,-160,0]+joint_ori2*(180)/np.pi,
    [60.542,40.878,5.210,-12.674,160.057,-160.012,-45]+joint_ori2*(180)/np.pi,
])
t=[0,0.5,1,2]
q3=q3/180*np.pi
Robot.control(q3,t)
qs=Robot.generateCurve(q3,t)
# qs=np.array(qs).T
# draw(qs)
# time.sleep(5)
# Robot.stop()

100


In [26]:
import numpy as np
import sympy as sp

theta=sp.symbols('theta_1:7')

DH = sp.Matrix(
    [[-sp.pi/2, 0, 1000, theta[0]],
     [sp.pi/2, 0, 1000, theta[1]],
     [0, 5200, 1000, -sp.pi/2+theta[2]],
     [0, 0, 1000, theta[3]]
    ]

)

the,alp,a,d=sp.symbols('theta alpha a d')
T=sp.Matrix([
    [sp.cos(the),-sp.sin(the),0,a],
    [sp.sin(the)*sp.cos(alp),sp.cos(the)*sp.cos(alp),-sp.sin(alp),-sp.sin(alp)*d],
    [sp.sin(the)*sp.sin(alp),sp.cos(the)*sp.sin(alp),sp.cos(alp),sp.cos(alp)*d],
    [0,0,0,1]
])
T

Matrix([
[           cos(theta),           -sin(theta),           0,             a],
[sin(theta)*cos(alpha), cos(alpha)*cos(theta), -sin(alpha), -d*sin(alpha)],
[sin(alpha)*sin(theta), sin(alpha)*cos(theta),  cos(alpha),  d*cos(alpha)],
[                    0,                     0,           0,             1]])

In [30]:
T_sym=[]
for i in range(4):
    T_sym+=[T.subs([(alp,DH[i,0]),(a,DH[i,1]),(d,DH[i,2]),(the,DH[i,3])])]


In [31]:
T123=sp.simplify(T_sym[0]*T_sym[1]*T_sym[2]*T_sym[3])
T123

Matrix([
[ sin(theta_2 + theta_3 + theta_4)*cos(theta_1),  cos(theta_1)*cos(theta_2 + theta_3 + theta_4), sin(theta_1),  3000*sin(theta_1) + 5200*cos(theta_1)*cos(theta_2)],
[             -cos(theta_2 + theta_3 + theta_4),               sin(theta_2 + theta_3 + theta_4),            0,                            5200*sin(theta_2) + 1000],
[-sin(theta_1)*sin(theta_2 + theta_3 + theta_4), -sin(theta_1)*cos(theta_2 + theta_3 + theta_4), cos(theta_1), -5200*sin(theta_1)*cos(theta_2) + 3000*cos(theta_1)],
[                                             0,                                              0,            0,                                                   1]])